In [80]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from dataprep.clean import clean_country, clean_df, validate_lat_long, clean_headers
import json

In [81]:
food_pr_df = pd.read_csv("Food_production.csv")

In [82]:
food_pr_df.head(10)

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,...,371.076923,482.4,0.945482,1.907692,2.897446,7.60,5.846154,18786.2,14450.92308,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.80,3.943662,49576.3,69825.77465,13449.891480
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.3,4.754098,...,347.647059,59.1,0.628415,2.705882,1.202186,0.88,5.176471,2754.2,16201.17647,3762.568306
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0,0.9,0.708419,...,NaN,0.0,1.355236,14.666667,1.858316,1.81,20.111111,0.0,NaN,NaN
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0,2.6,4.820513,...,NaN,620.1,0.911681,NaN,0.581197,2.04,NaN,16438.6,NaN,4683.361823
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0,1.4,1.541311,...,NaN,217.7,0.515670,NaN,0.521368,1.83,NaN,9493.3,NaN,2704.643875
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0,1.6,5.008798,...,203.503036,435.7,0.524927,0.836058,4.565982,15.57,7.272303,22477.4,10498.55208,NaN


CLEANING DATA


In [83]:
food_pr_df = clean_headers(food_pr_df, case='snake')

to_drop = [
       'eutrophying_emissions_per_1000kcal_g_p_oeq_per_1000kcal',
       'eutrophying_emissions_per_100g_protein_g_p_oeq_per_100_grams_protein',
       'freshwater_withdrawals_per_1000kcal_liters_per_1000kcal',
       'freshwater_withdrawals_per_100g_protein_liters_per_100g_protein',
       'greenhouse_gas_emissions_per_1000kcal_kg_c_oeq_per_1000kcal',
       'greenhouse_gas_emissions_per_100g_protein_kg_c_oeq_per_100g_protein',
       'land_use_per_1000kcal_m_per_1000kcal',
       'land_use_per_100g_protein_m_per_100g_protein',
       'scarcity_weighted_water_use_per_100g_protein_liters_per_100g_protein',
       'scarcity_weighted_water_use_per_1000kcal_liters_per_1000_kilocalories'
       ]
food_pr_df.drop(to_drop, axis=1, inplace=True)

# Dropping every row that has a single NaN value
food_pr_df.dropna(how='any', inplace=True)
# Checking the negative values for the land_use_change columns as seen in df.describe()
cond = food_pr_df['land_use_change'] < 0
food_pr_df.loc[cond, :]

Column Headers Cleaning Report:
	23 values cleaned (100.0%)


,food_product,land_use_change,animal_feed,farm,processing,transport,packging,retail,total_emissions,eutrophying_emissions_per_kilogram_g_p_oeq_per_kilogram,freshwater_withdrawals_per_kilogram_liters_per_kilogram,land_use_per_kilogram_m_per_kilogram,scarcity_weighted_water_use_per_kilogram_liters_per_kilogram
11,Nuts,-2.1,0.0,2.1,0.0,0.1,0.1,0.0,0.2,19.15,4133.8,12.96,229889.8
19,Olive Oil,-0.4,0.0,4.3,0.7,0.5,0.9,0.0,6.0,37.26,2141.8,26.31,177480.2
25,Citrus Fruit,-0.1,0.0,0.3,0.0,0.1,0.0,0.0,0.3,2.24,82.7,0.86,4662.7
29,Wine,-0.1,0.0,0.6,0.1,0.1,0.7,0.0,1.4,4.57,78.9,1.78,1149.3


In [84]:
cond=food_pr_df['food_product'].isin(['Beef (beef herd)', 'Lamb & Mutton', 'Cheese', 'Beef (dairy herd)', 'Dark Chocolate', 'Coffee', 'Palm Oil', 'Pig Meat', 'Poultry Meat', 'Olive Oil'])
df = food_pr_df.loc[cond, 'food_product':'total_emissions'].sort_values(by='total_emissions', ascending=False)
df = df.drop(columns='total_emissions')
df = df.set_index('food_product')
df = df.div(df.sum(axis=1), axis=0)*100
df = df.reset_index()

# Colors
colors = ["#067528", "#725428", "#b8615c", "#1a53ff", "#36454F", "#913D94", "#00DCF0"]

# Graph 1
fig = px.bar(
    df[:5], 
    y=['farm', 'land_use_change', 'animal_feed',  'processing', 'transport', 'packging', 'retail'], 
    x='food_product', 
    title="CO2 emissions",
    color_discrete_sequence=colors,
    barmode='group',
    orientation='v',
    template='simple_white',
    labels={
        'variable':'Step Di Produzione'}
)
# Traces 1
fig.update_traces(
    texttemplate="%{y:.0f}%", 
    textposition='outside',
    )

# Layout 1
fig.update_layout(
    width=1500,
    height=650,
    title = 'Top 5 alimenti per contribuzione di emissione per categoria (%)',     
    title_x = 0.5,     
    title_y = 0.9,     
    title_xanchor='center',     
    title_yanchor='top',
    
    yaxis = dict(
        title=None,
        ticksuffix='%',
        range=[0,100]),
    
    xaxis = dict(
        title=None),
    uniformtext_minsize=6, 
    uniformtext_mode='hide',
    legend=dict(x=0.95),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(
        # family="Courier New, monospace",
        size=15,  # Set the font size here
    ),
)
newnames ={ 'farm':"Coltivazione", 'land_use_change':'Utilizzo di terreno', 'animal_feed':'Alimenti per animali', 
          'processing':'Processo', 'transport':'Trasporti', 'packging':'Packaging', 'retail':'Rivendita'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

# Visualization
fig.show()



In [85]:

# save the graph in a json file
with open('graph.json', 'w') as f:
    json.dump(json.loads(fig.to_json()), f)

In [86]:
# print("var farm =", df['farm'].tolist())
# print("var land_use_change =",df['land_use_change'].tolist())
# print("var animal_feed =",df['animal_feed'].tolist())
# print("var processing =",df['processing'].tolist())
# print("var transport =",df['transport'].tolist())
# print("var packging =",df['packging'].tolist())
# print("var retail =",df['retail'].tolist())
# print("var food_product =",df['food_product'].tolist())

